# 役割
本番運用時の予測を行うnotebook。モデルの学習（`train.ipynb`）が完了している必要がある。

# インポート

In [ ]:
import preprocessing
from feature_producing_prediction import PredictionFeatureCreator
import predictioning
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [221]:
%autoreload

# 事前準備
**当日出走馬が確定した時点**で実行できる

In [201]:
# 予測時の学習母集団はcommon/data/prediction_population/に配置している
POPULATION_DIR = preprocessing.COMMON_DATA_DIR / "prediction_population"

In [202]:
# 当日出走馬の過去成績テーブルの前処理
horse_results_preprocessed = preprocessing.process_horse_results(
    population_dir=POPULATION_DIR,
    populaton_filename="population.csv",
    input_filename="horse_results_prediction.csv",
    output_filename="horse_results_prediction.csv"
)

In [203]:
# データの重複チェック
horse_results_preprocessed.duplicated(subset=[ "horse_id", "date"]).sum()


np.int64(0)

In [204]:
# 当日出走馬の血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds(
    population_dir=POPULATION_DIR,
    populaton_filename= "population.csv",
    input_filename="peds_prediction.csv",
    output_filename="peds_prediction.csv"
)

In [216]:
pfc = PredictionFeatureCreator()
# 過去成績集計は事前に行うことができる
pfc.create_baselog()
pfc.agg_horse_n_races()
pfc.agg_horse_n_races_relative()
pfc.agg_interval()

agg_horse_n_races_relative:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_interval()...


In [206]:
pfc.baselog

,date,race_id,horse_id,date_horse,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,win,rentai,show,place
2842,2025-03-02,202510011201,2022103128,2025-02-23,2.0,220.0,0.5,3.0,0,1700,0,1.0,12,106.1,0,1,1,10.0
2966,2025-03-02,202510011203,2022102890,2025-02-23,3.0,112.0,1.3,0.0,0,1000,0,1.0,14,60.4,0,0,1,10.0
273,2025-03-02,202506020207,2022101206,2025-02-23,10.0,0.0,1.5,0.0,1,1600,0,2.0,13,96.5,0,0,0,5.0
3411,2025-03-02,202510011206,2020100621,2025-02-23,12.0,0.0,1.0,1.0,1,1800,0,5.0,14,109.0,0,0,0,10.0
3437,2025-03-02,202510011206,2021106904,2025-02-23,5.0,80.0,0.8,1.0,1,2000,0,2.0,12,120.9,0,0,0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,2025-03-02,202506020211,2016106606,2018-11-24,11.0,0.0,1.3,0.0,1,1800,0,1.0,18,110.9,0,0,0,5.0
1107,2025-03-02,202506020211,2016106606,2018-11-11,10.0,0.0,1.0,0.0,1,1800,0,1.0,15,110.8,0,0,0,5.0
1108,2025-03-02,202506020211,2016106606,2018-11-04,8.0,0.0,2.6,1.0,0,1600,0,1.0,9,101.5,0,0,0,5.0
1109,2025-03-02,202506020211,2016106606,2018-09-22,13.0,0.0,5.2,1.0,0,1800,2,1.0,15,118.5,0,0,0,6.0


# 当日の予測処理
レース直前出走直前に実行する

In [226]:
pfc.race_info

,race_id,race_type,around,course_len,weather,ground_state,race_class,place
0,202506020210,1,0,1200,0,0,4,6


In [227]:
# 特徴量の更新
features = pfc.create_features(
    race_id="202510011209",  # 予測するレースidを指定
    skip_agg_horse=True  # 事前に集計した場合はスキップできる
)

fetching shubuta page html...


c:\Users\Onoe Daichi\Downloads\競馬AI\keiba_ai\v3_exp_4\src\feature_producing_prediction.py:184: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...
running agg_sire()...
running cross_feature()...
merging all features...


In [228]:
# 予測
prediction = predictioning.predict(
    features,
    model_filename="model_latest.pkl",
    calibration_model_filename="calibration_model_latest.pkl",
    # sort_col="expect_return",
)

In [229]:
# 2024年までを学習させたバージョン
sorted_features = prediction.sort_values(by='expect_return', ascending=False)
sorted_features

,race_id,umaban,tansyo_odds,popularity,pred,pred_calibrated,expect_return,expect_return_calibrated
14,202510011209,15,19.6,6,0.055341,0.043140,1.084677,0.845545
10,202510011209,11,397.3,18,0.002448,0.000000,0.972555,0.000000
6,202510011209,7,94.2,11,0.009245,0.005450,0.870833,0.513351
9,202510011209,10,12.4,5,0.069095,0.049180,0.856782,0.609836
8,202510011209,9,74.8,10,0.011325,0.005450,0.847100,0.407629
11,202510011209,12,9.9,4,0.085075,0.071823,0.842245,0.711050
3,202510011209,4,22.0,7,0.037403,0.042553,0.822857,0.936170
1,202510011209,2,31.8,8,0.025497,0.025510,0.810816,0.811224
7,202510011209,8,4.9,2,0.164517,0.158730,0.806132,0.777778
5,202510011209,6,1.9,1,0.419334,0.510638,0.796735,0.970213


In [ ]:
# 2023年までを学習させたバージョン
sorted_features = prediction.sort_values(by='expect_return', ascending=False)
sorted_features

,race_id,umaban,tansyo_odds,popularity,pred,pred_calibrated,expect_return,expect_return_calibrated
6,202510011011,8,32.0,12,0.032647,0.035714,1.044707,1.142857
0,202510011011,2,6.6,4,0.128622,0.149660,0.848902,0.987755
12,202510011011,14,17.9,9,0.045309,0.042537,0.811023,0.761418
4,202510011011,6,17.6,8,0.045701,0.042537,0.804338,0.748657
1,202510011011,3,4.4,1,0.181561,0.189189,0.798868,0.832432
9,202510011011,11,6.4,3,0.121159,0.113636,0.775419,0.727273
5,202510011011,7,5.5,2,0.136506,0.149660,0.750785,0.823129
3,202510011011,5,10.3,6,0.072848,0.065574,0.750331,0.675410
7,202510011011,9,9.1,5,0.076981,0.065574,0.700531,0.596721
8,202510011011,10,12.7,7,0.054466,0.042537,0.691719,0.540224
